In [1]:
# basically like a binary tree, but inserts at left-most position, every position is filled
# heapify means creating a binary heap out of a binary tree

import os
import sys
import re
project_path = re.match('/.+interview-practice/', os.getcwd())[0]
sys.path.append(project_path)

from operator import attrgetter
from collections import deque
from functions.custom_errors import NodeNotFoundError
# from functions.structs.tree import Node, BinaryHeap, binary_heap_from_array

In [2]:
def binary_heap_from_array(array):
    heap = BinaryHeap(Node(array[0]))
    for item in array[1:]:
        heap.insert(Node(item))
    return heap

In [3]:
class Node:
    def __init__(self, data=None):
        self.data = data
        self.left = None
        self.right = None

class BinaryHeap:
    """Make sure to only add Node objects to the Binary Heap
    
    Example 1: Using insert only
    source: https://www.tutorialspoint.com/python_data_structure/python_tree_traversal_algorithms.htm
            2
         /    \
       10      9
      /  \    /
     5    6  1
    
    heap = BinaryHeap(Node(2))
    heap.insert(Node(10))
    heap.insert(Node(9))
    heap.insert(Node(5))
    heap.insert(Node(6))
    heap.insert(Node(1))
    """
    
    def __init__(self, root=None):
        self.root = root
            
    def find_parent(self, curr_node):
        for parent_node in self.iterative_inorder():
            if (parent_node.left == curr_node) or parent_node.right == curr_node:
                return parent_node
        
    def swap(self, curr_node, child_node):
        """swap links on curr and child nodes
        update root if swap
        """
        if curr_node == self.root:
            self.root = child_node
        
        if curr_node.left == child_node:
            curr_node.right, child_node.right = child_node.right, curr_node.right
            curr_node.left, child_node.left = child_node.left, curr_node
            
        elif curr_node.right == child_node:
            curr_node.left, child_node.left = child_node.left, curr_node.left
            curr_node.right, child_node.right = child_node.right, curr_node

    def heapsort(self):
        """Only swaps one-layer deep
        Traverses from bottom up in postorder
        
        In the future: try to recurse down and place the new node at the right spot
        For now, can just keep rerunning this, it will eventually sort itself
        """
        
        for curr_node in self.recursive_postorder():
            try:
                print(curr_node.data, prev_node.data)
            except:
                print(curr_node.data, None)
            
            max_node = max([node for node in [curr_node, curr_node.left, curr_node.right] if node is not None],
                           key=attrgetter("data"))
            
            if max_node == curr_node.left:
                
                parent_node = self.find_parent(curr_node)
                if parent_node:
                    if parent_node.left == curr_node:
                        parent_node.left = curr_node.left
                    elif parent_node.right == curr_node:
                        parent_node.right = curr_node.left

                print('swap left', curr_node.data, curr_node.left.data)
                self.swap(curr_node, curr_node.left)
                    
            elif max_node == curr_node.right:
                
                parent_node = self.find_parent(curr_node)
                if parent_node:
                    if parent_node.left == curr_node:
                        parent_node.left = curr_node.right
                    elif parent_node.right == curr_node:
                        parent_node.right = curr_node.right

                print('swap right', curr_node.data, curr_node.right.data)
                self.swap(curr_node, curr_node.right)

            else:
                pass
                    
            prev_node = curr_node            
                    

    def insert(self, new_node):
        """level order insert
        See: https://www.geeksforgeeks.org/insertion-in-a-binary-tree-in-level-order/
        """
        if self.root is None:
            self.root = new_node
        
        for node in self.iterative_levelorder():
            if not node.left:
                node.left = new_node
                break

            if not node.right:
                node.right = new_node
                break
            
    def recursive_inorder(self, node='default'):
        """traverse recursively
        left -> root -> right
        """
        
        if node=='default':
            node = self.root

        if node.left:
            yield from self.recursive_inorder(node.left)
            
        yield node
        
        if node.right:
            yield from self.recursive_inorder(node.right)
            
    def recursive_postorder(self, node='default'):
        """traverse recursively
        left -> right -> root
        """
        
        if node=='default':
            node = self.root
        
        if node.left:
            yield from self.recursive_postorder(node.left)
        
        if node.right:
            yield from self.recursive_postorder(node.right)
            
        yield node
        
    def iterative_levelorder(self, node='default'):
        """traverse iteratively
        add current level elements to stack and pop them to return
        """
        
        stack = deque()
        stack.append(self.root)

        while stack:
            node = stack.popleft()
            yield node

            if node.left is not None:
                stack.append(node.left)

            if node.right is not None:
                stack.append(node.right)
                
    def iterative_inorder(self, node='default'):
        """See: https://www.geeksforgeeks.org/inorder-tree-traversal-without-recursion/
        
        1) Create an empty stack.
        2) Initialize current node as root
        3) Push the current node to S and set current = current->left until current is NULL
        4) If current is NULL and stack is not empty then 
             a) Pop the top item from stack.
             b) Print the popped item, set current = popped_item->right 
             c) Go to step 3.
        5) If current is NULL and stack is empty then we are done.
        
        Example 1:
        add 27, add 14, add 10, pop 10, return 10, node.right=None
        pop 14, return 14, node.right=19
        add 19, pop 19, return 19, node.right=None
        pop 27, return 27, node.right=35
        add 35, add 31, pop 31, return 31, node.right=None
        pop 35, return 35, node.right=42
        add 42, pop 42, return 42, node.right=None
        """
        
        if node=='default':
            node = self.root
            
        stack = deque()
        
        while True:
            # during first loop, adds all left nodes to stack
            # during subsequent loops, may add if node.right exists
            while node is not None:
                stack.append(node)
                node = node.left
            
            if stack:
                node = stack.pop()
                yield node
                node = node.right  # adds node.right to stack if it exists
                
            else:
                break

    def print_tree(self, order='inorder'):
        i = 0
        if order == 'inorder':
            
            for node in self.recursive_inorder():
                print(node.data)
                i += 1
                if i == 100:
                    break
                
        if order == 'levelorder':
            for node in self.iterative_levelorder():
                print(node.data)
                i += 1
                if i == 100:
                    break

In [4]:
a = Node(2)
b = Node(10)
c = Node(9)
d = Node(5)
e = Node(6)
f = Node(1)

heap = BinaryHeap(a)
heap.insert(b)
heap.insert(c)
heap.insert(d)
heap.insert(e)
heap.insert(f)

In [5]:
heap.heapsort()

5 None
6 5
10 6
1 10
9 1
2 9
swap left 2 10


In [6]:
heap.print_tree("levelorder")

10
2
9
5
6
1


In [7]:
heap.heapsort()

5 None
6 5
2 6
swap right 2 6
1 2
9 1
10 9


In [8]:
heap.print_tree("levelorder")

10
6
9
5
2
1


In [9]:
heap.root.data

10

In [10]:
heap.root.left.data

6

In [11]:
heap.root.left.left.data

5

In [12]:
heap.root.left.right.data

2

In [13]:
heap.root.right.data

9

In [14]:
heap.root.right.left.data

1